## XGBoost

[XGBoost: A Scalable Tree Boosting System](https://www.kdd.org/kdd2016/papers/files/rfp0697-chenAemb.pdf) 的论文可以多读几遍，不仅仅是算法上的优化，更多的是工程实现上。在算法使用泰勒级数进似优化函数，用到了一阶和二阶梯度，给出寻找划分的方法和叶子结点权重更新的公式。另外，还有以下改进：

1. 划分点的寻找方法，单机版可以选择穷举的贪心算法来穷举，但对于分布式（数据无法一次性加载到内存中）给出一个用分位数 (percentile) 近似的算法。
2. 对缺失值的处理，通过对缺失设置默认路径可以提高性能。（主要思想是：利用非缺失值来对缺失值样本进行划分）
3. 对大数据量进行存储分成压缩列块(compressed column block)，并行训练机制；
4. 并用线程来实现对存储块的缓存机制 (Cache-Aware Access)。
5. 为了降低硬盘的读取耗时，提供了两种方案，
    - 数据按列压缩; 
    - Block sharding (块碎片化，通过将大数据分散到多个硬盘上）。
    
    
下对是论文的导读。可以根据李航书中的三个步骤来看，即：模型，策略，算法。


### 2. Tree Boosting in a Nutshell

#### 2.1 模型

XGBoost 所基于的模型也还是 GBM，是一个集成模型，其基学习器是(CART 回归树），模型可以描述为：

给定一个数据集，n 个样本，每个样本有 m 个属性，即：$ D = \{(x_i, y_i)\}, |D| = n, x_i \in \mathbb{R}^m, y_i \in \mathbb{R}$

$$
\hat y_i = \emptyset (x_i) = \sum_{k=1}^{K}f_k(x_i),  f_k \in \mathcal{F} \tag{1}
$$

其中， $\mathcal{F} = \{f(x) = w_{q(x)}\} ，( q: \mathbb{R}^m \to T, w \in \mathbb{R}^T) $。

- q 是基学习器即回归树的结构参数。
- w 是树上叶子节点的权重。
- T 是树的叶子节点数。

最终的模型是所有树对应叶子权重的累加和。


#### 2.2 策略

策略就是最小化下面的损失函数了。

$$
\mathcal{L}(\emptyset) = \sum_{i}L(y_i, \hat y_i) + \sum_{k} \Omega(f_k) \\ \tag{2}
\text{where} \space\space \Omega(f_k) = \gamma T + \frac{1}{2}\lambda||w||^2
$$

其中对于叶子节点个数 $T$ 及叶子节点的权重 $w$，我们可能需要这两类参数进行下惩罚正则，防止过拟合。

我们提升算法递推公式为：$ y_i^{(t)} = y_i^{(t - 1)} + f_t(x_i)$, 公式 $(2)$ 可以写成下面的形式，即第 $t$ 次迭代的损失：

$$
\mathcal{L}^{(t)} = \sum_{i}L(y_i, \hat y_i^{(t - 1)} + f_t(x_i)) +  \Omega(f_t) 
$$

其中，
- $\hat y_i^{(t - 1)} $ 累加学习到的模型。
- $f_t(x_i)$ 根据残差新学习到的模型。
- $\Omega(f_t)$, 对新学习到模型的惩罚，即对叶子节点数，叶子权重做正则。（因为是迭代公式，上面的迭代的学习器已经处理过正则了。）

根据 [Friedman:Additive logistic regression: a statistical view of boosting. Annals of Statistics, 28(2)](https://projecteuclid.org/download/pdf_1/euclid.aos/1016218223) ，上式可以用二阶导数级数替换。即：

$$
\mathcal{L}^{(t)} \simeq  \sum_{i=1}^{n} [L(y_i, \hat y_i^{(t-1)}) + g_i f_t(x_i) + \frac{1}{2}h_i f_t^2(x_i)] + \Omega(f_t)
$$

基中：

- $g_i = \partial_{\hat y^{(t-1)}} L(y_i, \hat y_i^{(t-1)})$ , 即损失函数的一阶梯度
- $h_i = \partial^2_{\hat y^{(t-1)}} L(y_i, \hat y_i^{(t-1)})$,  即损失函数的二阶梯度

上式把常数项去掉，则有：

$$
\tilde{\mathcal{L}}^{(t)} =  \sum_{i=1}^{n} [g_i f_t(x_i) + \frac{1}{2}h_i f_t^2(x_i)] + \Omega(f_t) \tag{3}
$$

> 泰勒公式展开的推导过程如下：
首先，假设 $f(x)$ 在 $x=x_0$ 处有任意阶导数，则幂级数泰勒公式为：

$$
\sum_{n = 0}^{\infty} \frac{f^{(n)}(x_0)}{n!} (x - x_0)^n = f(x_0) + f'(x_0)(x-x_0)+ f''(x_0)(x-x_0)^2 + \cdots
$$
> 考虑函数 $L(y_i, \hat y_i^{(t-1)} +  f_t(x_i)) $, 则其在 $x_0 = y_i^{(t-1)} -  f_t(x_i)$ 处的二级展开式如下：

$$
L(y_i, y_i^{(t-1)}) + g_i f_t(x_i) + \frac{1}{2} h_i f_t^2(x_i)
$$

> 其中第一项为常数项，我们可以在优化目标中去掉，只保留后面与 $f_t(x_i)$ 相关的两项。所以可以得到我们最终的优化目标。



##### 2.1 对公式 (3) 求解

公式 (3) 无正则项的形式如下：
$$
\tilde{\mathcal{L}}^{(t)} =  \sum_{i=1}^{n} [g_i f_t(x_i) + \frac{1}{2}h_i f_t^2(x_i)] 
$$

公式所表述的是所有样本在 $f_t$ 模型与其对应一、二阶导数($g_i, h_i$)相乘的累加和。上面公式可以换一个描述，即将对各个样本的描述，换成对各个叶子结点上的描述。

因为 $f_t$ 上有 $T$ 个叶子，每个叶子有一个对应的权重参数 $w_j$, 而每个样本仅属于一个叶子。所以对于属于叶子 $j$ 上的所有样本（记为：$I_j = \{i|q(X_i) = j\}$），则该叶子上损失量为(该叶子上所有样本的一、二阶梯度和乘上相应的 w_j 参数)，即：

$$
w_j\sum_{i \in I_j}g_i + \frac{1}{2} w_j^2 \sum_{i \in I_j}h_i
$$

则对于 $T$ 个叶子，其损失量为：

$$
\sum_{j=1}^T \big( w_j\sum_{i \in I_j}g_i + \frac{1}{2} w_j^2 \sum_{i \in I_j}h_i \big) \tag{4}
$$


将上式(4) 代入 $(3)$  重写，并且展开 $\Omega$ ：

\begin{align}
\tilde{\mathcal{L}}^{(t)} &=  \sum_{i=1}^{n} [g_i f_t(x_i) + \frac{1}{2}h_i f_t^2(x_i)] + \gamma T + \frac{1}{2}\lambda \sum_{j=1}^{T} w_j^2 \\
&= \sum_{j=1}^T \big( w_j\sum_{i \in I_j}g_i + \frac{1}{2} w_j^2 \sum_{i \in I_j}h_i \big) + \gamma T + \frac{1}{2}\lambda \sum_{j=1}^{T} w_j^2 \\
&= \sum_{j=1}^{T}[(\sum_{i \in I_j} g_i)w_j + \frac{1}{2}(\sum_{i \in I_j} h_i + \lambda)w_j^2] + \gamma T
\tag{5}
\end{align}

通过对上式 (5) 求导，求得其驻点，

$$
\frac{\partial{\tilde{\mathcal{L}}^{(t)}}}{\partial{w_j}} = \sum_{i \in I_j} g_i + (\sum_{i \in I_j} h_i + \lambda)w_j = 0
$$

可以求出最优解 $w_j^*$ 

$$
w_j^* = -\frac{\sum_{i \in I_j} g_i}{\sum_{i \in I_j} h_i + \lambda} \tag{6}
$$

将 $(6)$ 式代入 $(5)$ 中，可以对应的最小的损失为：

$$
\tilde{\mathcal{L}}^{(t)}(q) = -\frac{1}{2} \sum_{j=1}^{T} \frac{(\sum_{i \in I_j} g_i)^2}{\sum_{i \in I_j} h_i + \lambda} + \lambda T  \tag{7}
$$


我们树模型在划分时，我可以根据 $(7)$ 来找到最优的划分，假设划分出的两个集合为 $I_L, I_R$：

$$
\mathcal{L}_{split} = \frac{1}{2}[\frac{(\sum_{i \in I_L} g_i)^2}{\sum_{i \in I_L} h_i + \lambda} + \frac{(\sum_{i \in I_R} g_i)^2}{\sum_{i \in I_R} h_i + \lambda} - \frac{(\sum_{i \in I} g_i)^2}{\sum_{i \in I} h_i + \lambda}] \tag{8}
$$

有了 $(6), (8)$ 那么我们模型的策略就已经定了。

### 2.3 Shrinkage and column sampling

两种正则的化的方法，

1. Shrinkage 即对生成树使用学习因子，来进行正则。
2. 生成树使用随机森林的思想，在生成时随机对特征进行抽样。


### 3. Splitting Finding Algorithm 寻找最优划分算法

#### 3.1 贪心穷举
#### 3.2 分位数近似
#### 3.4 稀疏数据集处理（生成默认路径）

### 4. 系统设计

#### 1. Column Block for parallel computing
#### 2. Cache-aware access
#### 3. Blocks for out-of-core Computation
1. Block compression
2. Block sharding